In [5]:
#for pretty print to all vars
from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all" #default="last_expr"

#to_display_any_htmls
from IPython.core.display import display, HTML

In [6]:
StorParam=dict()
StorParam['hparams']=dict()
StorParam['metrics']=dict()
StorParam['filesMisClf']=dict()

In [7]:
#Change dir to read dataset files from zip's extract
import os
os.chdir('C:/Users/mihir.parikh/PyNb/wk_data_op/')
files=list()
categories=[fl for fl in os.listdir('.') if os.path.isdir(fl)]
for folder in categories:
        files+=[folder+'/'+p for p in os.listdir(folder)]
print(categories," Total files read:",len(files))

['ADD', 'BUY', 'EPG']  Total files read: 710


In [8]:
#preprocess file's text_contents before laoding to pandas dataframe
import re
dataF={'content':[],'catg':[]}
newfiles=[]
#adding to dataFrame after preprocessing:
#    [remove 1st few not null lines, remove files of less than 10 chars, merge concurrent spaces lines,etc]
skip1stLines=2
for fi in files:
    with open(fi,encoding="utf8") as f:
        #print("for file",fi)
        PREPROC=f.read()
        #make non-space similar to space
        PREPROC=PREPROC.replace(' ',' ') #care...left side wala asli space nai hai
        while PREPROC.count('  '):PREPROC=PREPROC.replace('  ',' ')
        while PREPROC.count('\n '):PREPROC=PREPROC.replace('\n ','\n')
        while PREPROC.count(' \n'):PREPROC=PREPROC.replace(' \n','\n')
        while PREPROC.count('\n\n'):PREPROC=PREPROC.replace('\n\n','\n')
        PREPROC=' '.join(PREPROC.split('\n')[0 if fi.startswith('Others') else skip1stLines:])
        #PREPROC=re.sub(r"\d+(\.\d*)?", r"",PREPROC) #remove all numbers (digits)
        PREPROC=re.sub(r"[(](\S*?)[)]", r"",PREPROC) #remove all b/w (brackets)
        PREPROC=re.sub(r"[^a-zA-Z ]", r"",PREPROC) #remove all non alpha (brackets)
        while PREPROC.count('  '):PREPROC=PREPROC.replace('  ',' ')
        if len(PREPROC)<10:continue
        dataF['content'].append(PREPROC)
        dataF['catg'].append(fi.split('/')[-2])
        newfiles.append(fi)
skippedfiles=list(set(files)-set(newfiles))
print(len(skippedfiles),"files ignored. New Dataset has",len(newfiles),"files.")
##check skip files of some catg
#sorted(list(filter((lambda st:st.startswith('MC')),skippedfiles)))

StorParam['files']=files
StorParam['newfiles']=newfiles
StorParam['preproc_comment']="default"

skippedfiles

4 files ignored. New Dataset has 706 files.


['BUY/reg-level_19CSR-RS30-61-105_The Day Care Provider and Other Day Care Personnel.xml.txt',
 'EPG/45,911.pdf.txt',
 'EPG/171006_Protections.pdf.txt',
 'EPG/45,910.pdf.txt']

In [9]:
#load to pandas dataframe and have a peek on each catg
import pandas as pd
df = pd.DataFrame.from_dict(dataF)
print(df['catg'].value_counts())
for k in df['catg'].value_counts().keys():display(HTML(df[(df['catg']==k)].head(5).to_html()))

StorParam['catg_val_count']=df['catg'].value_counts()

EPG    291
ADD    261
BUY    154
Name: catg, dtype: int64


,catg,content
415,EPG,Wasbtngton October MEMORANDUM FOR ALL COMPONEN...
416,EPG,UNITED STATES DISTRICT COURT FOR THE DISTRICT ...
417,EPG,UNITED STATES COURT OF APPEALS FOR THE NINTH C...
418,EPG,UNITED STATES COURT OF APPEALS FOR THE NINTH C...
419,EPG,United States Court of Appeals For the Seventh...


,catg,content
0,ADD,MIDDLE DISTRICT OF FLORIDA TAMPA DIVISION KENN...
1,ADD,UNITED STATES DISTRICT COURT EASTERN DISTRICT...
2,ADD,DISTRICT OF MINNESOTA James Chavira Plaintiff ...
3,ADD,UNITED STATES COURT OF APPEALS TENTH CIRCUIT K...
4,ADD,DISTRICT OF KANSAS Equal Employment Opportunit...


,catg,content
261,BUY,CFR The average unweighted amount and average...
262,BUY,IN THE SUPREME COURT OF PENNSYLVANIA WESTERN D...
263,BUY,IN THE SUPREME COURT OF PENNSYLVANIA WESTERN D...
264,BUY,RELATES TO KRS Chapter STATUTORY AUTHORITY KRS...
265,BUY,January Term FILED June No released at pm RORY...


In [10]:
#partial import of sklearn. will full load sub-modules as needed.
import sklearn

#split data for training and testing
from sklearn.model_selection import train_test_split
X=df['content']
Y=df['catg']
SplitRatios=(.5,.3,.2) #train test cv
#SplitRatio_ts=0.4
SplitRatio_ts=sum(SplitRatios[1:])#1-SplitRatios[0]
#SplitRatio_cv=0.5
SplitRatio_cv=SplitRatios[-1]/SplitRatio_ts


Random_ts=42
Random_cv=420
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, test_size=SplitRatio_ts,random_state=Random_ts)
X_test,X_cv,Y_test,Y_cv=sklearn.model_selection.train_test_split(X_test,Y_test,test_size=SplitRatio_cv,random_state=Random_cv)
print("Train",[X_train.shape,Y_train.shape],"Test",[X_test.shape,Y_test.shape],"CV",[X_cv.shape,Y_cv.shape])
#now data is ready
StorParam['SplitRatio_ts']=SplitRatio_ts
StorParam['SplitRatio_cv']=SplitRatio_cv
StorParam['Random_ts']=Random_ts
StorParam['Random_cv']=Random_cv

Train [(353,), (353,)] Test [(211,), (211,)] CV [(142,), (142,)]


### Multinomial NAIVE BAYES

In [11]:
#pipelining
from sklearn.pipeline import Pipeline
algoname='nb'
#naive_bayes
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

text_clf_nb = Pipeline([('vect', CountVectorizer(stop_words='english',ngram_range=(1,3))),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB(alpha=0.01))
                    ])
clfname='text_clf_'+algoname
StorParam['hparams'][algoname]={x:globals()[clfname].named_steps[x].get_params() for x in globals()[clfname].named_steps.keys()}
#fit_transforms of data to pipeline
fit_nb=text_clf_nb.fit(X_train,Y_train)

#testing model
trscr=text_clf_nb.score(X_train,Y_train)
tsscr=text_clf_nb.score(X_test,Y_test)
print("--> Accuracy")
print("TRAIN",trscr)
print("TEST",tsscr)
StorParam['metrics'][algoname]={'train':trscr,'test':tsscr,'cv':None}
StorParam['filesMisClf'][algoname]={'train':list(),'test':list(),'cv':list()}
#misClassifications
print("--> Train MisClassification")
TRpredicted_nb = text_clf_nb.predict(X_train)
misClTr_nb=list()
#sklearn.metrics.confusion_matrix
display(HTML(pd.crosstab(Y_train,TRpredicted_nb,rownames=["True"],colnames=["Train_Predicted"],margins=True).to_html()))
for i in range(len(TRpredicted_nb)):
    if TRpredicted_nb[i]!=Y_train.iloc[i]:
        print("X_Train.values[",i,"] Predicted",TRpredicted_nb[i],"Actual",Y_train.iloc[i],
              "File:[errI="+str(X_train.index[i]),newfiles[X_train.index[i]],"]")
        misClTr_nb.append(i)
        StorParam['filesMisClf'][algoname]['train'].append(newfiles[X_train.index[i]])
print(len(misClTr_nb)," misses in TrainData")
print("--> Test MisClassification")
TSpredicted_nb = text_clf_nb.predict(X_test)
misClTs_nb=list()
#sklearn.metrics.confusion_matrix
display(HTML(pd.crosstab(Y_test,TSpredicted_nb,rownames=["True"],colnames=["Test_Predicted"],margins=True).to_html()))
for i in range(len(TSpredicted_nb)):
    if TSpredicted_nb[i]!=Y_test.iloc[i]:
        print("X_test.values[",i,"] Predicted",TSpredicted_nb[i],"Actual",Y_test.iloc[i],
              "File:[errI="+str(X_test.index[i]),newfiles[X_test.index[i]],"]")
        misClTs_nb.append(i)
        StorParam['filesMisClf'][algoname]['test'].append(newfiles[X_test.index[i]])
print(len(misClTs_nb)," misses in TestData")


--> Accuracy
TRAIN 1.0
TEST 0.8957345971563981
--> Train MisClassification


Train_Predicted,ADD,BUY,EPG,All
True,,,,
ADD,120,0,0,120
BUY,0,69,0,69
EPG,0,0,164,164
All,120,69,164,353


0  misses in TrainData
--> Test MisClassification


Test_Predicted,ADD,BUY,EPG,All
True,,,,
ADD,82,0,7,89
BUY,0,41,8,49
EPG,7,0,66,73
All,89,41,81,211


X_test.values[ 13 ] Predicted EPG Actual ADD File:[errI=149 ADD/17-120.pdf.txt ]
X_test.values[ 31 ] Predicted ADD Actual EPG File:[errI=587 EPG/45752.pdf.txt ]
X_test.values[ 37 ] Predicted EPG Actual BUY File:[errI=382 BUY/RocheleauElder.pdf.txt ]
X_test.values[ 45 ] Predicted ADD Actual EPG File:[errI=692 EPG/45868.pdf.txt ]
X_test.values[ 46 ] Predicted EPG Actual ADD File:[errI=179 ADD/17-150.pdf.txt ]
X_test.values[ 58 ] Predicted EPG Actual BUY File:[errI=284 BUY/BuenoLR.pdf.txt ]
X_test.values[ 59 ] Predicted EPG Actual ADD File:[errI=133 ADD/17-103.pdf.txt ]
X_test.values[ 67 ] Predicted EPG Actual ADD File:[errI=78 ADD/17-043.pdf.txt ]
X_test.values[ 95 ] Predicted EPG Actual ADD File:[errI=163 ADD/17-134.pdf.txt ]
X_test.values[ 96 ] Predicted EPG Actual BUY File:[errI=338 BUY/MaddenMidland.pdf.txt ]
X_test.values[ 101 ] Predicted EPG Actual ADD File:[errI=136 ADD/17-106.pdf.txt ]
X_test.values[ 104 ] Predicted ADD Actual EPG File:[errI=674 EPG/45849.pdf.txt ]
X_test.values[

In [12]:
#study misclassified doc errI [by filename or errI_index]
#errI=newfiles.index('MC-275/0001553350-17-000313-4.txt')
errI=35
#print(errI,"TESTdata" if errI in X_test else "TRAINdata",newfiles[errI],X_test.get(errI,0) or X_train.get(errI,0))
#os.system("np++ "+newfiles[errI])

### SVM SVC [C-Support Vector Classification]

In [13]:
#pipelining
from sklearn.pipeline import Pipeline
algoname='svm'
#svm_svc
from sklearn.svm import SVC
text_clf_svm = Pipeline([('vect', CountVectorizer(stop_words='english',ngram_range=(1,3), )),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SVC(decision_function_shape="ovo", C = 1000000, kernel='rbf', gamma ='auto'))
                    ])

clfname='text_clf_'+algoname
StorParam['hparams'][algoname]={x:globals()[clfname].named_steps[x].get_params() for x in globals()[clfname].named_steps.keys()}
#fit_transforms of data to pipeline
fit_svm=text_clf_svm.fit(X_train,Y_train)

#testing model
trscr=text_clf_svm.score(X_train,Y_train)
tsscr=text_clf_svm.score(X_test,Y_test)
print("--> Accuracy")
print("TRAIN",trscr)
print("TEST",tsscr)
StorParam['metrics'][algoname]={'train':trscr,'test':tsscr,'cv':None}
StorParam['filesMisClf'][algoname]={'train':list(),'test':list(),'cv':list()}

#misClassifications
print("--> Train MisClassification")
TRpredicted_svm = text_clf_svm.predict(X_train)
misClTr_svm=list()
#sklearn.metrics.confusion_matrix
display(HTML(pd.crosstab(Y_train,TRpredicted_svm,rownames=["True"],colnames=["Train_Predicted"],margins=True).to_html()))
for i in range(len(TRpredicted_svm)):
    if TRpredicted_svm[i]!=Y_train.iloc[i]:
        print("X_Train.values[",i,"] Predicted",TRpredicted_svm[i],"Actual",Y_train.iloc[i],
              "File:[errI="+str(X_train.index[i]),newfiles[X_train.index[i]],"]")
        misClTr_svm.append(i)
        StorParam['filesMisClf'][algoname]['train'].append(newfiles[X_train.index[i]])
print(len(misClTr_svm)," misses in TrainData")

print("--> Test MisClassification")
TSpredicted_svm = text_clf_svm.predict(X_test)
misClTs_svm=list()
#sklearn.metrics.confusion_matrix
display(HTML(pd.crosstab(Y_test,TSpredicted_svm,rownames=["True"],colnames=["Test_Predicted"],margins=True).to_html()))
for i in range(len(TSpredicted_svm)):
    if TSpredicted_svm[i]!=Y_test.iloc[i]:
        print("X_test.values[",i,"] Predicted",TSpredicted_svm[i],"Actual",Y_test.iloc[i],
              "File:[errI="+str(X_test.index[i]),newfiles[X_test.index[i]],"]")
        misClTs_svm.append(i)
        StorParam['filesMisClf'][algoname]['test'].append(newfiles[X_test.index[i]])

print(len(misClTs_svm)," misses in TestData")


--> Accuracy
TRAIN 1.0
TEST 0.933649289099526
--> Train MisClassification


Train_Predicted,ADD,BUY,EPG,All
True,,,,
ADD,120,0,0,120
BUY,0,69,0,69
EPG,0,0,164,164
All,120,69,164,353


0  misses in TrainData
--> Test MisClassification


Test_Predicted,ADD,BUY,EPG,All
True,,,,
ADD,84,0,5,89
BUY,0,44,5,49
EPG,4,0,69,73
All,88,44,79,211


X_test.values[ 13 ] Predicted EPG Actual ADD File:[errI=149 ADD/17-120.pdf.txt ]
X_test.values[ 31 ] Predicted ADD Actual EPG File:[errI=587 EPG/45752.pdf.txt ]
X_test.values[ 37 ] Predicted EPG Actual BUY File:[errI=382 BUY/RocheleauElder.pdf.txt ]
X_test.values[ 45 ] Predicted ADD Actual EPG File:[errI=692 EPG/45868.pdf.txt ]
X_test.values[ 46 ] Predicted EPG Actual ADD File:[errI=179 ADD/17-150.pdf.txt ]
X_test.values[ 58 ] Predicted EPG Actual BUY File:[errI=284 BUY/BuenoLR.pdf.txt ]
X_test.values[ 95 ] Predicted EPG Actual ADD File:[errI=163 ADD/17-134.pdf.txt ]
X_test.values[ 127 ] Predicted EPG Actual BUY File:[errI=319 BUY/GraySeterus.pdf.txt ]
X_test.values[ 134 ] Predicted EPG Actual BUY File:[errI=344 BUY/McMahonLVNV.pdf.txt ]
X_test.values[ 152 ] Predicted EPG Actual ADD File:[errI=247 ADD/17-218.pdf.txt ]
X_test.values[ 174 ] Predicted ADD Actual EPG File:[errI=428 EPG/45,882.pdf.txt ]
X_test.values[ 178 ] Predicted EPG Actual ADD File:[errI=108 ADD/17-073.pdf.txt ]
X_test

In [14]:
#study misclassified doc errI [by filename or errI_index]
#errI=newfiles.index('BUY/Gallego.pdf.txt')
errI=0
#print(errI,"TESTdata" if errI in X_test else "TRAINdata",newfiles[errI],X_test.get(errI,0) or X_train.get(errI,0))
#os.system("np++ "+newfiles[errI])

### Decision Tree


In [15]:
#pipelining
from sklearn.pipeline import Pipeline
algoname='dt'
#DT
from sklearn.tree import DecisionTreeClassifier
text_clf_dt = Pipeline([('vect', CountVectorizer(stop_words='english',ngram_range=(1,3), )),
                     ('tfidf', TfidfTransformer()),
                     ('clf',DecisionTreeClassifier(
                         criterion='gini', splitter='best', max_depth=None, 
                         min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0,
                         max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0,
                         min_impurity_split=None, class_weight=None, presort=False))
                    ])

clfname='text_clf_'+algoname
StorParam['hparams'][algoname]={x:globals()[clfname].named_steps[x].get_params() for x in globals()[clfname].named_steps.keys()}
#fit_transforms of data to pipeline
fit_dt=text_clf_dt.fit(X_train,Y_train)

#testing model
trscr=text_clf_dt.score(X_train,Y_train)
tsscr=text_clf_dt.score(X_test,Y_test)
print("--> Accuracy")
print("TRAIN",trscr)
print("TEST",tsscr)
StorParam['metrics'][algoname]={'train':trscr,'test':tsscr,'cv':None}
StorParam['filesMisClf'][algoname]={'train':list(),'test':list(),'cv':list()}

#misClassifications
print("--> Train MisClassification")
TRpredicted_dt = text_clf_dt.predict(X_train)
misClTr_dt=list()
#sklearn.metrics.confusion_matrix
display(HTML(pd.crosstab(Y_train,TRpredicted_dt,rownames=["True"],colnames=["Train_Predicted"],margins=True).to_html()))
for i in range(len(TRpredicted_dt)):
    if TRpredicted_dt[i]!=Y_train.iloc[i]:
        print("X_Train.values[",i,"] Predicted",TRpredicted_dt[i],"Actual",Y_train.iloc[i],
              "File:[errI="+str(X_train.index[i]),newfiles[X_train.index[i]],"]")
        misClTr_dt.append(i)
        StorParam['filesMisClf'][algoname]['train'].append(newfiles[X_train.index[i]])

print(len(misClTr_dt)," misses in TrainData")

print("--> Test MisClassification")
TSpredicted_dt = text_clf_dt.predict(X_test)
misClTs_dt=list()
#sklearn.metrics.confusion_matrix
display(HTML(pd.crosstab(Y_test,TSpredicted_dt,rownames=["True"],colnames=["Test_Predicted"],margins=True).to_html()))
for i in range(len(TSpredicted_dt)):
    if TSpredicted_dt[i]!=Y_test.iloc[i]:
        print("X_test.values[",i,"] Predicted",TSpredicted_dt[i],"Actual",Y_test.iloc[i],
              "File:[errI="+str(X_test.index[i]),newfiles[X_test.index[i]],"]")
        misClTs_dt.append(i)
        StorParam['filesMisClf'][algoname]['test'].append(newfiles[X_test.index[i]])

print(len(misClTs_dt)," misses in TestData")


--> Accuracy
TRAIN 1.0
TEST 0.8436018957345972
--> Train MisClassification


Train_Predicted,ADD,BUY,EPG,All
True,,,,
ADD,120,0,0,120
BUY,0,69,0,69
EPG,0,0,164,164
All,120,69,164,353


0  misses in TrainData
--> Test MisClassification


Test_Predicted,ADD,BUY,EPG,All
True,,,,
ADD,76,1,12,89
BUY,1,41,7,49
EPG,11,1,61,73
All,88,43,80,211


X_test.values[ 5 ] Predicted EPG Actual ADD File:[errI=260 ADD/18-010.pdf.txt ]
X_test.values[ 19 ] Predicted EPG Actual BUY File:[errI=353 BUY/NCS678.docx.txt ]
X_test.values[ 31 ] Predicted ADD Actual EPG File:[errI=587 EPG/45752.pdf.txt ]
X_test.values[ 34 ] Predicted EPG Actual ADD File:[errI=231 ADD/17-202.pdf.txt ]
X_test.values[ 38 ] Predicted EPG Actual ADD File:[errI=88 ADD/17-053.pdf.txt ]
X_test.values[ 45 ] Predicted BUY Actual EPG File:[errI=692 EPG/45868.pdf.txt ]
X_test.values[ 54 ] Predicted EPG Actual ADD File:[errI=31 ADD/17 092.pdf.txt ]
X_test.values[ 60 ] Predicted EPG Actual ADD File:[errI=174 ADD/17-145.pdf.txt ]
X_test.values[ 64 ] Predicted ADD Actual EPG File:[errI=687 EPG/45863.pdf.txt ]
X_test.values[ 78 ] Predicted EPG Actual ADD File:[errI=234 ADD/17-205.pdf.txt ]
X_test.values[ 81 ] Predicted EPG Actual BUY File:[errI=381 BUY/RIS456.docx.txt ]
X_test.values[ 91 ] Predicted ADD Actual EPG File:[errI=485 EPG/45,941.pdf.txt ]
X_test.values[ 102 ] Predicted A

In [16]:
#study misclassified doc errI [by filename or errI_index]
#errI=newfiles.index('MC-275/0001553350-17-000313-4.txt')
errI=121
#print(errI,"TESTdata" if errI in X_test else "TRAINdata",newfiles[errI],X_test.get(errI,0) or X_train.get(errI,0))
#os.system("np++ "+newfiles[errI])

### Linear Standard Grad. Descent

In [17]:
#pipelining
from sklearn.pipeline import Pipeline
algoname='sgd'
#sgd
from sklearn.linear_model import SGDClassifier
text_clf_sgd = Pipeline([('vect', CountVectorizer(stop_words='english',ngram_range=(1,3), )),
                     ('tfidf', TfidfTransformer()),
                     ('clf',SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, random_state=42,
                                           max_iter=5, tol=None))
                    ])

clfname='text_clf_'+algoname
StorParam['hparams'][algoname]={x:globals()[clfname].named_steps[x].get_params() for x in globals()[clfname].named_steps.keys()}
#fit_transforms of data to pipeline
fit_sgd=text_clf_sgd.fit(X_train,Y_train)

#testing model
trscr=text_clf_sgd.score(X_train,Y_train)
tsscr=text_clf_sgd.score(X_test,Y_test)
print("--> Accuracy")
print("TRAIN",trscr)
print("TEST",tsscr)
StorParam['metrics'][algoname]={'train':trscr,'test':tsscr,'cv':None}
StorParam['filesMisClf'][algoname]={'train':list(),'test':list(),'cv':list()}

#misClassifications
print("--> Train MisClassification")
TRpredicted_sgd = text_clf_sgd.predict(X_train)
misClTr_sgd=list()
#sklearn.metrics.confusion_matrix
display(HTML(pd.crosstab(Y_train,TRpredicted_sgd,rownames=["True"],colnames=["Train_Predicted"],margins=True).to_html()))
for i in range(len(TRpredicted_sgd)):
    if TRpredicted_sgd[i]!=Y_train.iloc[i]:
        print("X_Train.values[",i,"] Predicted",TRpredicted_sgd[i],"Actual",Y_train.iloc[i],
              "File:[errI="+str(X_train.index[i]),newfiles[X_train.index[i]],"]")
        misClTr_sgd.append(i)
        StorParam['filesMisClf'][algoname]['train'].append(newfiles[X_train.index[i]])

print(len(misClTr_sgd)," misses in TrainData")

print("--> Test MisClassification")
TSpredicted_sgd = text_clf_sgd.predict(X_test)
misClTs_sgd=list()
#sklearn.metrics.confusion_matrix
display(HTML(pd.crosstab(Y_test,TSpredicted_sgd,rownames=["True"],colnames=["Test_Predicted"],margins=True).to_html()))
for i in range(len(TSpredicted_sgd)):
    if TSpredicted_sgd[i]!=Y_test.iloc[i]:
        print("X_test.values[",i,"] Predicted",TSpredicted_sgd[i],"Actual",Y_test.iloc[i],
              "File:[errI="+str(X_test.index[i]),newfiles[X_test.index[i]],"]")
        misClTs_sgd.append(i)
        StorParam['filesMisClf'][algoname]['test'].append(newfiles[X_test.index[i]])

print(len(misClTs_sgd)," misses in TestData")


--> Accuracy
TRAIN 1.0
TEST 0.9289099526066351
--> Train MisClassification


Train_Predicted,ADD,BUY,EPG,All
True,,,,
ADD,120,0,0,120
BUY,0,69,0,69
EPG,0,0,164,164
All,120,69,164,353


0  misses in TrainData
--> Test MisClassification


Test_Predicted,ADD,BUY,EPG,All
True,,,,
ADD,85,0,4,89
BUY,0,43,6,49
EPG,5,0,68,73
All,90,43,78,211


X_test.values[ 13 ] Predicted EPG Actual ADD File:[errI=149 ADD/17-120.pdf.txt ]
X_test.values[ 31 ] Predicted ADD Actual EPG File:[errI=587 EPG/45752.pdf.txt ]
X_test.values[ 37 ] Predicted EPG Actual BUY File:[errI=382 BUY/RocheleauElder.pdf.txt ]
X_test.values[ 45 ] Predicted ADD Actual EPG File:[errI=692 EPG/45868.pdf.txt ]
X_test.values[ 46 ] Predicted EPG Actual ADD File:[errI=179 ADD/17-150.pdf.txt ]
X_test.values[ 58 ] Predicted EPG Actual BUY File:[errI=284 BUY/BuenoLR.pdf.txt ]
X_test.values[ 67 ] Predicted EPG Actual ADD File:[errI=78 ADD/17-043.pdf.txt ]
X_test.values[ 95 ] Predicted EPG Actual ADD File:[errI=163 ADD/17-134.pdf.txt ]
X_test.values[ 127 ] Predicted EPG Actual BUY File:[errI=319 BUY/GraySeterus.pdf.txt ]
X_test.values[ 134 ] Predicted EPG Actual BUY File:[errI=344 BUY/McMahonLVNV.pdf.txt ]
X_test.values[ 143 ] Predicted EPG Actual BUY File:[errI=367 BUY/PedigoRobertson.pdf.txt ]
X_test.values[ 174 ] Predicted ADD Actual EPG File:[errI=428 EPG/45,882.pdf.txt ]

In [18]:
#study misclassified doc errI [by filename or errI_index]
#errI=newfiles.index('Others-275/new 206.txt')
errI=121
#print(errI,"TESTdata" if errI in X_test else "TRAINdata",newfiles[errI],X_test.get(errI,0) or X_train.get(errI,0))
#os.system("np++ "+newfiles[errI])

## CrossValidation
Beware: this notebook has opposite meaning of Test and CrossVal

And thats not cool. but meh, im lazy

In [19]:
print("--> NAIVE BAYES CrossVal MisClassification")
algoname='nb'
cvscr=text_clf_nb.score(X_cv,Y_cv)
print("TEST",algoname,cvscr)
StorParam['metrics'][algoname]['cv']=cvscr
CVpredicted_nb = text_clf_nb.predict(X_cv)
misClcv_nb=list()
#sklearn.metrics.confusion_matrix
display(HTML(pd.crosstab(Y_cv,CVpredicted_nb,rownames=["True"],colnames=["CrVal_Predicted"],margins=True).to_html()))
for i in range(len(CVpredicted_nb)):
    if CVpredicted_nb[i]!=Y_cv.iloc[i]:
        print("X_cv.values[",i,"] Predicted",CVpredicted_nb[i],"Actual",Y_cv.iloc[i],
              "File:[errI="+str(X_cv.index[i]),newfiles[X_cv.index[i]],"]")
        misClcv_nb.append(i)
        StorParam['filesMisClf'][algoname]['cv'].append(newfiles[X_cv.index[i]])
print(len(misClcv_nb)," misses in CrossValData")

--> NAIVE BAYES CrossVal MisClassification
TEST nb 0.9084507042253521


CrVal_Predicted,ADD,BUY,EPG,All
True,,,,
ADD,47,0,5,52
BUY,1,28,7,36
EPG,0,0,54,54
All,48,28,66,142


X_cv.values[ 1 ] Predicted EPG Actual ADD File:[errI=18 ADD/16189.pdf.txt ]
X_cv.values[ 21 ] Predicted EPG Actual ADD File:[errI=131 ADD/17-101.pdf.txt ]
X_cv.values[ 24 ] Predicted EPG Actual BUY File:[errI=320 BUY/Hageman.pdf.txt ]
X_cv.values[ 25 ] Predicted EPG Actual ADD File:[errI=141 ADD/17-111.pdf.txt ]
X_cv.values[ 50 ] Predicted ADD Actual BUY File:[errI=375 BUY/RielNavient.pdf.txt ]
X_cv.values[ 58 ] Predicted EPG Actual ADD File:[errI=257 ADD/18-007.pdf.txt ]
X_cv.values[ 62 ] Predicted EPG Actual BUY File:[errI=321 BUY/HamburgerNorth.pdf.txt ]
X_cv.values[ 73 ] Predicted EPG Actual BUY File:[errI=324 BUY/HudsonCiti.pdf.txt ]
X_cv.values[ 97 ] Predicted EPG Actual ADD File:[errI=142 ADD/17-112.pdf.txt ]
X_cv.values[ 105 ] Predicted EPG Actual BUY File:[errI=289 BUY/CFPBGordon.pdf.txt ]
X_cv.values[ 119 ] Predicted EPG Actual BUY File:[errI=296 BUY/DennisReizman.pdf.txt ]
X_cv.values[ 131 ] Predicted EPG Actual BUY File:[errI=363 BUY/ORS 205_460 - Order to show cause why in

In [20]:
print("--> SVM CrossVal MisClassification")
algoname='svm'
cvscr=text_clf_svm.score(X_cv,Y_cv)
print("TEST",algoname,cvscr)
StorParam['metrics'][algoname]['cv']=cvscr
CVpredicted_svm = text_clf_svm.predict(X_cv)
misClcv_svm=list()
#sklearn.metrics.confusion_matrix
display(HTML(pd.crosstab(Y_cv,CVpredicted_svm,rownames=["True"],colnames=["CrVal_Predicted"],margins=True).to_html()))
for i in range(len(CVpredicted_svm)):
    if CVpredicted_svm[i]!=Y_cv.iloc[i]:
        print("X_cv.values[",i,"] Predicted",CVpredicted_svm[i],"Actual",Y_cv.iloc[i],
              "File:[errI="+str(X_cv.index[i]),newfiles[X_cv.index[i]],"]")
        misClcv_svm.append(i)
        StorParam['filesMisClf'][algoname]['cv'].append(newfiles[X_cv.index[i]])
print(len(misClcv_svm)," misses in CrossValData")

--> SVM CrossVal MisClassification
TEST svm 0.9225352112676056


CrVal_Predicted,ADD,BUY,EPG,All
True,,,,
ADD,46,0,6,52
BUY,0,32,4,36
EPG,1,0,53,54
All,47,32,63,142


X_cv.values[ 1 ] Predicted EPG Actual ADD File:[errI=18 ADD/16189.pdf.txt ]
X_cv.values[ 2 ] Predicted EPG Actual ADD File:[errI=65 ADD/17-030.pdf.txt ]
X_cv.values[ 21 ] Predicted EPG Actual ADD File:[errI=131 ADD/17-101.pdf.txt ]
X_cv.values[ 25 ] Predicted EPG Actual ADD File:[errI=141 ADD/17-111.pdf.txt ]
X_cv.values[ 58 ] Predicted EPG Actual ADD File:[errI=257 ADD/18-007.pdf.txt ]
X_cv.values[ 62 ] Predicted EPG Actual BUY File:[errI=321 BUY/HamburgerNorth.pdf.txt ]
X_cv.values[ 97 ] Predicted EPG Actual ADD File:[errI=142 ADD/17-112.pdf.txt ]
X_cv.values[ 105 ] Predicted EPG Actual BUY File:[errI=289 BUY/CFPBGordon.pdf.txt ]
X_cv.values[ 115 ] Predicted ADD Actual EPG File:[errI=551 EPG/45716.pdf.txt ]
X_cv.values[ 119 ] Predicted EPG Actual BUY File:[errI=296 BUY/DennisReizman.pdf.txt ]
X_cv.values[ 131 ] Predicted EPG Actual BUY File:[errI=363 BUY/ORS 205_460 - Order to show cause why invalid claim of encumbrance should not be stricken - 2013 Oregon Revised Statutes.mht.txt ]


In [21]:
print("--> DecisionTree CrossVal MisClassification")
algoname='dt'
cvscr=text_clf_dt.score(X_cv,Y_cv)
print("TEST",algoname,cvscr)
StorParam['metrics'][algoname]['cv']=cvscr
CVpredicted_dt = text_clf_dt.predict(X_cv)
misClcv_dt=list()
#sklearn.metrics.confusion_matrix
display(HTML(pd.crosstab(Y_cv,CVpredicted_dt,rownames=["True"],colnames=["CrVal_Predicted"],margins=True).to_html()))
for i in range(len(CVpredicted_dt)):
    if CVpredicted_dt[i]!=Y_cv.iloc[i]:
        print("X_cv.values[",i,"] Predicted",CVpredicted_dt[i],"Actual",Y_cv.iloc[i],
              "File:[errI="+str(X_cv.index[i]),newfiles[X_cv.index[i]],"]")
        misClcv_dt.append(i)
        StorParam['filesMisClf'][algoname]['cv'].append(newfiles[X_cv.index[i]])
print(len(misClcv_dt)," misses in CrossValData")

--> DecisionTree CrossVal MisClassification
TEST dt 0.8309859154929577


CrVal_Predicted,ADD,BUY,EPG,All
True,,,,
ADD,42,0,10,52
BUY,0,32,4,36
EPG,8,2,44,54
All,50,34,58,142


X_cv.values[ 1 ] Predicted EPG Actual ADD File:[errI=18 ADD/16189.pdf.txt ]
X_cv.values[ 2 ] Predicted EPG Actual ADD File:[errI=65 ADD/17-030.pdf.txt ]
X_cv.values[ 8 ] Predicted EPG Actual ADD File:[errI=70 ADD/17-035.pdf.txt ]
X_cv.values[ 15 ] Predicted EPG Actual BUY File:[errI=342 BUY/McGillCiti.PDF.txt ]
X_cv.values[ 19 ] Predicted ADD Actual EPG File:[errI=701 EPG/MarshallRawlings042017.pdf.txt ]
X_cv.values[ 21 ] Predicted EPG Actual ADD File:[errI=131 ADD/17-101.pdf.txt ]
X_cv.values[ 23 ] Predicted ADD Actual EPG File:[errI=683 EPG/45858.pdf.txt ]
X_cv.values[ 36 ] Predicted BUY Actual EPG File:[errI=497 EPG/45,953.pdf.txt ]
X_cv.values[ 37 ] Predicted ADD Actual EPG File:[errI=429 EPG/45,883.pdf.txt ]
X_cv.values[ 43 ] Predicted EPG Actual BUY File:[errI=275 BUY/BenzemannCiti.pdf.txt ]
X_cv.values[ 56 ] Predicted ADD Actual EPG File:[errI=482 EPG/45,938.pdf.txt ]
X_cv.values[ 70 ] Predicted EPG Actual ADD File:[errI=110 ADD/17-075.pdf.txt ]
X_cv.values[ 72 ] Predicted ADD A

In [22]:
print("--> LinearStandardGradientDescent CrossVal MisClassification")
algoname='sgd'
cvscr=text_clf_sgd.score(X_cv,Y_cv)
print("TEST",algoname,cvscr)
StorParam['metrics'][algoname]['cv']=cvscr
CVpredicted_sgd = text_clf_sgd.predict(X_cv)
misClcv_sgd=list()
#sklearn.metrics.confusion_matrix
display(HTML(pd.crosstab(Y_cv,CVpredicted_sgd,rownames=["True"],colnames=["CrVal_Predicted"],margins=True).to_html()))
for i in range(len(CVpredicted_sgd)):
    if CVpredicted_sgd[i]!=Y_cv.iloc[i]:
        print("X_cv.values[",i,"] Predicted",CVpredicted_sgd[i],"Actual",Y_cv.iloc[i],
              "File:[errI="+str(X_cv.index[i]),newfiles[X_cv.index[i]],"]")
        misClcv_sgd.append(i)
        StorParam['filesMisClf'][algoname]['cv'].append(newfiles[X_cv.index[i]])
print(len(misClcv_sgd)," misses in CrossValData")

--> LinearStandardGradientDescent CrossVal MisClassification
TEST sgd 0.9154929577464789


CrVal_Predicted,ADD,BUY,EPG,All
True,,,,
ADD,45,0,7,52
BUY,0,32,4,36
EPG,1,0,53,54
All,46,32,64,142


X_cv.values[ 1 ] Predicted EPG Actual ADD File:[errI=18 ADD/16189.pdf.txt ]
X_cv.values[ 2 ] Predicted EPG Actual ADD File:[errI=65 ADD/17-030.pdf.txt ]
X_cv.values[ 21 ] Predicted EPG Actual ADD File:[errI=131 ADD/17-101.pdf.txt ]
X_cv.values[ 25 ] Predicted EPG Actual ADD File:[errI=141 ADD/17-111.pdf.txt ]
X_cv.values[ 58 ] Predicted EPG Actual ADD File:[errI=257 ADD/18-007.pdf.txt ]
X_cv.values[ 62 ] Predicted EPG Actual BUY File:[errI=321 BUY/HamburgerNorth.pdf.txt ]
X_cv.values[ 97 ] Predicted EPG Actual ADD File:[errI=142 ADD/17-112.pdf.txt ]
X_cv.values[ 105 ] Predicted EPG Actual BUY File:[errI=289 BUY/CFPBGordon.pdf.txt ]
X_cv.values[ 115 ] Predicted ADD Actual EPG File:[errI=551 EPG/45716.pdf.txt ]
X_cv.values[ 119 ] Predicted EPG Actual BUY File:[errI=296 BUY/DennisReizman.pdf.txt ]
X_cv.values[ 131 ] Predicted EPG Actual BUY File:[errI=363 BUY/ORS 205_460 - Order to show cause why invalid claim of encumbrance should not be stricken - 2013 Oregon Revised Statutes.mht.txt ]


------------------------------------------------------------------------------------------
#### Analysing StorParams

In [23]:
StorParam.keys()

dict_keys(['hparams', 'metrics', 'filesMisClf', 'files', 'newfiles', 'preproc_comment', 'catg_val_count', 'SplitRatio_ts', 'SplitRatio_cv', 'Random_ts', 'Random_cv'])

In [24]:
StorParam['metrics']

{'dt': {'cv': 0.8309859154929577, 'test': 0.8436018957345972, 'train': 1.0},
 'nb': {'cv': 0.9084507042253521, 'test': 0.8957345971563981, 'train': 1.0},
 'sgd': {'cv': 0.9154929577464789, 'test': 0.9289099526066351, 'train': 1.0},
 'svm': {'cv': 0.9225352112676056, 'test': 0.933649289099526, 'train': 1.0}}

In [25]:
#MisClassifiedFiles_alongwith_NumberOfTimeMisClassified
pd.Series(sum([sum(list(I.values()),[]) for I in StorParam['filesMisClf'].values()],[])).value_counts()


ADD/16189.pdf.txt                                                                                                                       4
EPG/45,882.pdf.txt                                                                                                                      4
EPG/45752.pdf.txt                                                                                                                       4
EPG/45868.pdf.txt                                                                                                                       4
ADD/17-101.pdf.txt                                                                                                                      4
BUY/GraySeterus.pdf.txt                                                                                                                 4
EPG/45732.pdf.txt                                                                                                                       4
ADD/17-112.pdf.txt                

# tinker

In [26]:
rsTr=text_clf_nb.predict_proba(X_train)
rs=text_clf_nb.predict_proba(X_test)

In [27]:
p=1
trs=(sum([(max(x))**p for x in rsTr])/len(rsTr))**(1/p)

trs

0.9999999884853356

In [28]:
print(text_clf_nb.classes_)
for i in misClTs_nb:
    if max(rs[i])<trs:#,"less by",-max(rs[i])+trs
        print(i,"chale",rs[i],"pr far from true=",max(rs[i])-rs[i][text_clf_nb.classes_.tolist().index(Y_test.iloc[i])])
for i in misClTs_nb:
    if max(rs[i])>trs:
        print(i,"Locha",rs[i],"extra by",max(rs[i])-trs,"pr far from true=",)
#chale=>no catg has enough confidence on them
#Locha=>wrong catg has confidence on them by <-->

['ADD' 'BUY' 'EPG']
13 chale [1.02993687e-02 2.37242605e-06 9.89698259e-01] pr far from true= 0.979398890155869
37 chale [3.26824980e-03 1.44337302e-04 9.96587413e-01] pr far from true= 0.9964430755996854
45 chale [0.92023571 0.00141035 0.07835395] pr far from true= 0.8418817622368489
46 chale [1.61757595e-03 1.38619304e-09 9.98382423e-01] pr far from true= 0.9967648467147093
58 chale [0.01828005 0.00954766 0.97217228] pr far from true= 0.9626246181351056
59 chale [2.60547518e-01 8.64564117e-15 7.39452482e-01] pr far from true= 0.4789049630853823
67 chale [3.88477651e-02 1.87371605e-08 9.61152216e-01] pr far from true= 0.9223044511416798
95 chale [3.50624770e-01 1.49692249e-08 6.49375215e-01] pr far from true= 0.2987504456482437
96 chale [0.00341831 0.41479692 0.58178477] pr far from true= 0.16698784877795714
101 chale [1.39553064e-01 5.22927386e-15 8.60446936e-01] pr far from true= 0.7208938728933804
104 chale [6.64512149e-01 1.69205771e-08 3.35487834e-01] pr far from true= 0.32902431

In [29]:
' '==' '

False

In [30]:
print('             precision    recall  f1-score   support')
for al in "nb svm dt sgd".split():
    for dsy,ds in {"train":"TR","test":"TS","cv":"CV"}.items():
        print(al.upper(),ds.lower(),'\n'.join(sklearn.metrics.classification_report(globals()["Y_"+dsy],globals()[ds+"predicted_"+al]).split('\n')[1:]))
#print("svm_ts",'\n'.join(sklearn.metrics.classification_report(Y_test,TSpredicted_svm).split('\n')[1:]))
#print("dt_ts",'\n'.join(sklearn.metrics.classification_report(Y_test,TSpredicted_dt).split('\n')[1:]))
#print("sgd_ts",'\n'.join(sklearn.metrics.classification_report(Y_test,TSpredicted_sgd).split('\n')[1:]))


             precision    recall  f1-score   support
NB tr 
        ADD       1.00      1.00      1.00       120
        BUY       1.00      1.00      1.00        69
        EPG       1.00      1.00      1.00       164

avg / total       1.00      1.00      1.00       353

NB ts 
        ADD       0.92      0.92      0.92        89
        BUY       1.00      0.84      0.91        49
        EPG       0.81      0.90      0.86        73

avg / total       0.90      0.90      0.90       211

NB cv 
        ADD       0.98      0.90      0.94        52
        BUY       1.00      0.78      0.88        36
        EPG       0.82      1.00      0.90        54

avg / total       0.92      0.91      0.91       142

SVM tr 
        ADD       1.00      1.00      1.00       120
        BUY       1.00      1.00      1.00        69
        EPG       1.00      1.00      1.00       164

avg / total       1.00      1.00      1.00       353

SVM ts 
        ADD       0.95      0.94      0.95        89
 

In [31]:
fit_nb

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 3), preprocessor=None, stop_words='english',
        ...near_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True))])

In [32]:
def fitPipe(algoname,text_clf_):
    global StorParam,X_train,X_test,Y_train,Y_test,X_cv,Y_cv
    StorParam['hparams'][algoname]={x:text_clf_.named_steps[x].get_params() for x in text_clf_.named_steps.keys()}
    #fit_transforms of data to pipeline
    fit_=text_clf_.fit(X_train,Y_train)

    #testing model
    print("--> Accuracy")
    
    TRpredicted_ = text_clf_.predict(X_train)
    #trscr_=text_clf_.score(X_train,Y_train)
    crtab_tr=pd.crosstab(Y_train,TRpredicted_,rownames=["True"],colnames=["Train_Predicted"],margins=True)
    trscr_=1-(crtab_tr.All.All-sum([crtab_tr[c][c] for c in text_clf_.classes_]))/crtab_tr.All.All
    print("TRAIN",trscr_)

    TSpredicted_ = text_clf_.predict(X_test)
    #tsscr_=text_clf_.score(X_test,Y_test)
    crtab_ts=pd.crosstab(Y_test,TSpredicted_,rownames=["True"],colnames=["Test_Predicted"],margins=True)
    tsscr_=1-(crtab_ts.All.All-sum([crtab_ts[c][c] for c in text_clf_.classes_]))/crtab_ts.All.All
    print("TEST",tsscr_)
    
    StorParam['metrics'][algoname]={'train':trscr_,'test':tsscr_,'cv':None}
    StorParam['filesMisClf'][algoname]={'train':list(),'test':list(),'cv':list()}

    #misClassifications
    print("--> Train MisClassification")
    misClTr_=list()
    #sklearn.metrics.confusion_matrix
    display(HTML(crtab_tr.to_html()))
    for i in range(len(TRpredicted_)):
        if TRpredicted_[i]!=Y_train.iloc[i]:
            print("X_Train.values[",i,"] Predicted",TRpredicted_[i],"Actual",Y_train.iloc[i],
                  "File:[errI="+str(X_train.index[i]),newfiles[X_train.index[i]],"]")
            misClTr_.append(i)
            StorParam['filesMisClf'][algoname]['train'].append(newfiles[X_train.index[i]])
    print(len(misClTr_)," misses in TrainData")

    print("--> Test MisClassification")
    misClTs_=list()
    #sklearn.metrics.confusion_matrix
    display(HTML(crtab_ts.to_html()))
    for i in range(len(TSpredicted_)):
        if TSpredicted_[i]!=Y_test.iloc[i]:
            print("X_test.values[",i,"] Predicted",TSpredicted_[i],"Actual",Y_test.iloc[i],
                  "File:[errI="+str(X_test.index[i]),newfiles[X_test.index[i]],"]")
            misClTs_.append(i)
            StorParam['filesMisClf'][algoname]['test'].append(newfiles[X_test.index[i]])

    print(len(misClTs_)," misses in TestData")
    
    #push these vars to global namespace
    returnD={k+algoname:v for k,v in locals().items() if k.endswith('_')}
    for k,v in returnD.items():globals()[k]=v

In [33]:
#pipelining
from sklearn.pipeline import Pipeline
#svm_svc
from sklearn.svm import SVC

text_clf_svm = Pipeline([('vect', CountVectorizer(stop_words='english',ngram_range=(1,3), )),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SVC(decision_function_shape="ovo", C = 1000000, kernel='rbf', gamma ='auto'))
                    ])
fitPipe('svm',text_clf_svm)


--> Accuracy
TRAIN 1.0
TEST 0.933649289099526
--> Train MisClassification


Train_Predicted,ADD,BUY,EPG,All
True,,,,
ADD,120,0,0,120
BUY,0,69,0,69
EPG,0,0,164,164
All,120,69,164,353


0  misses in TrainData
--> Test MisClassification


Test_Predicted,ADD,BUY,EPG,All
True,,,,
ADD,84,0,5,89
BUY,0,44,5,49
EPG,4,0,69,73
All,88,44,79,211


X_test.values[ 13 ] Predicted EPG Actual ADD File:[errI=149 ADD/17-120.pdf.txt ]
X_test.values[ 31 ] Predicted ADD Actual EPG File:[errI=587 EPG/45752.pdf.txt ]
X_test.values[ 37 ] Predicted EPG Actual BUY File:[errI=382 BUY/RocheleauElder.pdf.txt ]
X_test.values[ 45 ] Predicted ADD Actual EPG File:[errI=692 EPG/45868.pdf.txt ]
X_test.values[ 46 ] Predicted EPG Actual ADD File:[errI=179 ADD/17-150.pdf.txt ]
X_test.values[ 58 ] Predicted EPG Actual BUY File:[errI=284 BUY/BuenoLR.pdf.txt ]
X_test.values[ 95 ] Predicted EPG Actual ADD File:[errI=163 ADD/17-134.pdf.txt ]
X_test.values[ 127 ] Predicted EPG Actual BUY File:[errI=319 BUY/GraySeterus.pdf.txt ]
X_test.values[ 134 ] Predicted EPG Actual BUY File:[errI=344 BUY/McMahonLVNV.pdf.txt ]
X_test.values[ 152 ] Predicted EPG Actual ADD File:[errI=247 ADD/17-218.pdf.txt ]
X_test.values[ 174 ] Predicted ADD Actual EPG File:[errI=428 EPG/45,882.pdf.txt ]
X_test.values[ 178 ] Predicted EPG Actual ADD File:[errI=108 ADD/17-073.pdf.txt ]
X_test

In [34]:
1-m/t

NameError: name 'm' is not defined

In [48]:
sum([[(i,j) for j in range(i,4+1)] for i in range(1,3+1)],[])

[(1, 1), (1, 2), (1, 3), (1, 4), (2, 2), (2, 3), (2, 4), (3, 3), (3, 4)]

In [ ]:
#grid
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA, NMF
from sklearn.feature_selection import SelectKBest, chi2

pipe = Pipeline([
    ('reduce_dim', PCA()),
    ('classify', LinearSVC())
])
N_FEATURES_OPTIONS = [2, 4, 8]
C_OPTIONS = [1, 10, 100, 1000]
param_grid = [
    {
        'reduce_dim': [PCA(iterated_power=7), NMF()],
        'reduce_dim__n_components': N_FEATURES_OPTIONS,
        'classify__C': C_OPTIONS
    },
    {
        'reduce_dim': [SelectKBest(chi2)],
        'reduce_dim__k': N_FEATURES_OPTIONS,
        'classify__C': C_OPTIONS
    },
]
reducer_labels = ['PCA', 'NMF', 'KBest(chi2)']

grid = GridSearchCV(pipe, cv=3, n_jobs=1, param_grid=param_grid)


from sklearn.datasets import load_digits
digits = load_digits()
grid.fit(digits.data, digits.target)


import matplotlib.pyplot as plt
%matplotlib inline
mean_scores = np.array(grid.cv_results_['mean_test_score'])
# scores are in the order of param_grid iteration, which is alphabetical
mean_scores = mean_scores.reshape(len(C_OPTIONS), -1, len(N_FEATURES_OPTIONS))
# select score for best C
mean_scores = mean_scores.max(axis=0)
bar_offsets = (np.arange(len(N_FEATURES_OPTIONS)) *
               (len(reducer_labels) + 1) + .5)

plt.figure()
COLORS = 'bgrcmyk'
for i, (label, reducer_scores) in enumerate(zip(reducer_labels, mean_scores)):
    plt.bar(bar_offsets + i, reducer_scores, label=label, color=COLORS[i])

plt.title("Comparing feature reduction techniques")
plt.xlabel('Reduced number of features')
plt.xticks(bar_offsets + len(reducer_labels) / 2, N_FEATURES_OPTIONS)
plt.ylabel('Digit classification accuracy')
plt.ylim((0, 1))
plt.legend(loc='upper left')

In [ ]:
pipe.named_steps

In [ ]:
grid.best_params_

In [ ]:
grid.best_estimator_

In [ ]:
Y_digit_t_prd=grid.best_estimator_.predict(digits.data)

In [ ]:
sum(Y_digit_t_prd==digits.target)-len(Y_digit_t_prd)

In [ ]:
1-223/len(Y_digit_t_prd)

In [ ]:
grid.score(digits.data,digits.target)

In [ ]:
grid.best_score_

In [ ]:
grid.grid_scores_

# mah echo algo

In [38]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

class Echo(BaseEstimator, TransformerMixin):
    def transform(self, X, *_):return X
    def fit(self, *_):return self


In [45]:
text_clf_sgdE = Pipeline([('vect', CountVectorizer(stop_words='english',ngram_range=(1,3), )),
                    # ('tfidf', TfidfTransformer()),
                    ('echo',Echo()),
                     ('clf',SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, random_state=42,
                                           max_iter=5, tol=None))
                    ])
text_clf_sgdE.fit(X_train,Y_train)
print("futt")
print(text_clf_sgdE.score(X_test,Y_test))

fit: cpu was here
transform: cpu was here
futt
transform: cpu was here
0.909952606635071
